# Recommendation Walkthrough - Lead recommendation

This walkthrough will take you through a simple recommendation task. By the end of the tutorial, you should be able to create a simple machine learning workflow to recommendation leads based on associated entity properties.

## Create a project

To start, we'll quickly create a Squirro project that we can work in. To do this you'll need a running Squirro cluster and a valid API token.

In [ ]:
CLUSTER = ""
TOKEN = ""

# get a client
from squirro_client import SquirroClient
client = SquirroClient(client_id=None, client_secret=None, cluster=CLUSTER)
client.authenticate(refresh_token=TOKEN)

# create a project
PROJECT_ID = client.new_project("Recommendation Walkthrough").get("id")
print PROJECT_ID

## Loading pretrained models

Before we load any data, we'll upload 3 pretrained models to find `forward-looking interest` statements, and tag them with a `deal type` and `industry`.

### Load forward interest model

In [ ]:
import json
with open('./forward_interest/fasttext_quantized.json', 'rb') as f:
    forward_interest_workflow = json.load(f)
forward_interest_workflow_id = client.new_machinelearning_workflow(
    PROJECT_ID, name='forward_interest', config=forward_interest_workflow,
    ml_models='./forward_interest').get('id')
print forward_interest_workflow_id

### Load deal type model

In [ ]:
with open('./deal_type/tfidf_sgd/tfidf_sgd.json', 'rb') as f:
    deal_type_workflow = json.load(f)
deal_type_workflow_id = client.new_machinelearning_workflow(
    PROJECT_ID, name='deal_type', config=deal_type_workflow,
    ml_models='./deal_type/tfidf_sgd').get('id')
print deal_type_workflow_id

### Load industry model

In [ ]:
with open('./industry/fasttext_quantized.json', 'rb') as f:
    industry_workflow = json.load(f)
industry_workflow_id = client.new_machinelearning_workflow(
    PROJECT_ID, name='industry', config=industry_workflow,
    ml_models='./industry').get('id')
print industry_workflow_id

## Loading a tagging pipelet

Now that our ML models are on the server, we can use the Machine Learning Service to create entities and tag items on ingestion. For this task, we'll use a premade pipelet that first classifies each sentence as forward-looking or not. If a sentence is forward-looking, then we attempt to tag it with a deal type and industry. All 3 models will be applied with a threshold confidence that must be met to apply the tag.

### Editing pipelet configuration

Before using the pipelet, we have to change its configuration to point to our newly created models and project.

In [ ]:
with open('./pipelet_config.json', 'rb') as f:
    pipelet_config = json.load(f)
pipelet_config['ForwardInterestDealsPipelet']['config'] = {
    'cluster': CLUSTER,
    'token': TOKEN,
    'models_project_id': PROJECT_ID,
    'forward_interest_workflow_id': forward_interest_workflow_id,
    'forward_interest_threshold': 0.90,
    'deal_type_workflow_id': deal_type_workflow_id,
    'deal_type_threshold': 0.75,
    'industry_workflow_id': industry_workflow_id,
    'industry_threshold': 0.75
}
with open('./pipelet_config.json', 'wb') as f:
    json.dump(pipelet_config, f)

### Upload pipelet to Squirro

In [ ]:
!{"squirro_asset pipelet upload -c %s -t %s ./forward_interest_deals_pipelet.py forward_interest_deals" % (
    CLUSTER, TOKEN)}

## Loading data

The next step is to load data in our Squirro instance. We can now run a pre-made Squirro data loader script to insert our data set. The pipelet is doing quite a bit and running locally, so be patient :) :

In [ ]:
!{"./load.sh %s %s %s" % (CLUSTER, TOKEN, PROJECT_ID)}

## Setting up the dashboard

Now that all the data is in and tagged, we can make our dashboard. To make it, go to your dashboard, follow the video below:

In [ ]:
%%HTML
<video width="1025" height="768" controls>
  <source src="./recommendations_dashboard.mp4" type="video/mp4">
</video>

## Reset

WARNING: This deletes the project!!!

In [ ]:
client.delete_project(PROJECT_ID)